In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, MinMaxScaler, RobustScaler, LabelEncoder
from sklearn.impute import SimpleImputer
from sklearn.decomposition import PCA
from sklearn.linear_model import LinearRegression, LogisticRegression, LogisticRegressionCV
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.metrics import mean_absolute_error, mean_squared_error

from xgboost import XGBRegressor, XGBClassifier

from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import InputLayer, Dense
from tensorflow.keras.optimizers import Adam

from math import sqrt


In [2]:
df = pd.read_csv(r'C:/Users/Nycht/Documents/Programming/Project/MONAS Dashboard/Modelling/Data')
df = df.drop(['Date', 'UTC'], axis=1)
df = df.sample(n=75000, random_state=42) 

PermissionError: [Errno 13] Permission denied: 'C:/Users/Nycht/Documents/Programming/Project/MONAS Dashboard/Modelling/Data'

In [3]:
df.columns

Index(['lokasi', 'con_prec.mm.', 'mic_prec.mm.', 'sha_prec.mm.',
       'suhu2m.degC.', 'dew2m.degC.', 'rh2m...', 'wspeed.m.s.', 'wdir.deg.',
       'lcloud...', 'mcloud...', 'hcloud...', 'surpre.Pa.', 'clmix.kg.kg.',
       'wamix.kg.kg.', 'outlr.W.m2.', 'pblh.m.', 'lifcl.m.', 'cape.j.kg.',
       'mdbz', 't950.degC.', 'rh950...', 'ws950.m.s.', 'wd950.deg.',
       't800.degC.', 'rh800...', 'ws800.m.s.', 'wd800.deg.', 't500.degC.',
       'rh500...', 'ws500.m.s.', 'wd500.deg.', 'hari', 'LAT', 'LON', 'ELEV',
       'rh_obs'],
      dtype='object')

In [21]:
# import pandas as pd
# import seaborn as sns
# import numpy as np
# import matplotlib.pyplot as plt

# num_cols = 5
# num_rows = (len(df.columns) + num_cols - 1) // num_cols
# fig, axes = plt.subplots(num_rows, num_cols, figsize=(15, 15))

# for i, column in enumerate(df.columns):
#     row, col = divmod(i, num_cols)  
#     sns.histplot(data=df, x=column, kde=True, bins=50, ax=axes[row, col])
#     axes[row, col].set_title(f'Distribution of {column}')
#     axes[row, col].set_xlabel(column)
#     axes[row, col].set_ylabel('Frequency')

# for i in range(len(df.columns), num_rows * num_cols):
#     row, col = divmod(i, num_cols)
#     fig.delaxes(axes[row, col])

# plt.tight_layout()
# plt.show()


In [22]:
left_skewed_cols = ["dew2m.degC.", "rh2m...", "lcloud...", "surpre.Pa.", "wamix.kg.kg.", "outlr.W.m2.", "rh950...", "rh800...", "rh500..."]
for col in left_skewed_cols:
    df[col], _ = stats.boxcox(df[col] + 1)  

right_skewed_cols = ["wspeed.m.s.", "pblh.m.", "lifcl.m.",  "ws950.m.s.", "ws800.m.s.", "ws500.m.s.", "cape.j.kg.", "wd500.deg."]
for col in right_skewed_cols:
    df[col] = np.log(df[col])  

bimodal_cols = [ "wd800.deg.", "wd950.deg."]
num_bins = 4
for col in bimodal_cols:
    df[col] = pd.cut(df[col], bins=num_bins, labels=False)


/home/kkn03/.local/lib/python3.6/site-packages/numpy/core/_methods.py:205: RuntimeWarning: overflow encountered in multiply
  x = um.multiply(x, x, out=x)
/home/kkn03/.local/lib/python3.6/site-packages/numpy/core/_methods.py:216: RuntimeWarning: overflow encountered in reduce
  ret = umr_sum(x, axis, dtype, out, keepdims)
/home/kkn03/.local/lib/python3.6/site-packages/pandas/core/series.py:726: RuntimeWarning: divide by zero encountered in log
  result = getattr(ufunc, method)(*inputs, **kwargs)
/home/kkn03/.local/lib/python3.6/site-packages/pandas/core/series.py:726: RuntimeWarning: invalid value encountered in log
  result = getattr(ufunc, method)(*inputs, **kwargs)


In [23]:
def clean_dataset(df):
    assert isinstance(df, pd.DataFrame), "df needs to be a pd.DataFrame"
    df.dropna(inplace=True)
    indices_to_keep = ~df.isin([np.nan, np.inf, -np.inf]).any(axis=1)
    return df[indices_to_keep].astype(np.float64)

In [24]:
#Regression dataset

df = clean_dataset(df)
y = df["rh_obs"] 
X = df.drop(["rh_obs"], axis=1)

x_train, x_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

scaler = StandardScaler()
x_train = scaler.fit_transform(x_train)
x_test = scaler.transform(x_test)

In [66]:
from sklearn.preprocessing import StandardScaler
from sklearn.impute import SimpleImputer
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import Pipeline
from sklearn.cross_decomposition import PLSRegression

# Create a pipeline that includes imputation, scaling, and modeling
pipeline = Pipeline([
    ('imputer', SimpleImputer(strategy='mean')),
    ('model', XGBRegressor())
])

# Fit the pipeline
pipeline.fit(x_train, y_train)

# Score the model
score = pipeline.score(x_test, y_test)
y_pred = pipeline.predict(x_test)


In [67]:
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score

# Calculate regression metrics
mse = mean_squared_error(y_test, y_pred)
mae = mean_absolute_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

print("Mean Squared Error (MSE):", mse)
print("Mean Absolute Error (MAE):", mae)
print("R-squared (R2) Score:", r2)


Mean Squared Error (MSE): 42.783734234335824
Mean Absolute Error (MAE): 4.935417590715614
R-squared (R2) Score: 0.6963540037651934


In [54]:
from sklearn.model_selection import RandomizedSearchCV

# Define the hyperparameter grid
param_dist = {
    'model__n_components': [1, 2, 3, 4, 5, 6],  # Number of components
    'model__scale': [True, False],  # Whether to scale the input data
    # Add more hyperparameters as needed
}

# Create the RandomizedSearchCV object
random_search = RandomizedSearchCV(
    pipeline, param_distributions=param_dist, n_iter=20, cv=5, n_jobs=-1, random_state=42, scoring='r2'
)

# Fit the randomized search on your data
random_search.fit(x_train, y_train)

# Get the best hyperparameters and model
best_params = random_search.best_params_
best_model = random_search.best_estimator_

# Score the best model on the test data
score = best_model.score(x_test, y_test)
y_pred = best_model.predict(x_test)

# Print the best hyperparameters and the model's score
print("Best Hyperparameters:", best_params)
print("Best Model Score (R2):", score)


/home/kkn03/.local/lib/python3.6/site-packages/sklearn/model_selection/_search.py:289: UserWarning: The total space of parameters 12 is smaller than n_iter=20. Running 12 iterations. For exhaustive searches, use GridSearchCV.
  % (grid_size, self.n_iter, grid_size), UserWarning)


Best Hyperparameters: {'model__scale': False, 'model__n_components': 6}
Best Model Score (R2): 0.5761617282840776


In [55]:
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score

# Calculate regression metrics
mse = mean_squared_error(y_test, y_pred)
mae = mean_absolute_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

print("Mean Squared Error (MSE):", mse)
print("Mean Absolute Error (MAE):", mae)
print("R-squared (R2) Score:", r2)


Mean Squared Error (MSE): 61.136566396354
Mean Absolute Error (MAE): 6.088575211447852
R-squared (R2) Score: 0.5761617282840776


In [56]:
from sklearn.linear_model import ElasticNet
from sklearn.model_selection import RandomizedSearchCV
from sklearn.pipeline import Pipeline

# Create a pipeline that includes imputation, scaling, and modeling
pipeline = Pipeline([
    ('imputer', SimpleImputer(strategy='mean')),
    ('model', ElasticNet())
])

# Define the hyperparameters and their potential values
param_dist = {
    'model__alpha': [0.1, 1.0, 10.0],
    'model__l1_ratio': [0.1, 0.5, 0.9],
    'model__fit_intercept': [True, False],
    'model__normalize': [True, False]
}

# Create a RandomizedSearchCV object
random_search = RandomizedSearchCV(
    pipeline, param_distributions=param_dist, n_iter=10, cv=5, scoring='neg_mean_squared_error', n_jobs=-1
)

# Fit the random search on your data
random_search.fit(x_train, y_train)

# Get the best hyperparameters and model
best_params = random_search.best_params_
best_model = random_search.best_estimator_

# Score the best model on the test data
y_pred = best_model.predict(x_test)

# Print the best hyperparameters
print("Best Hyperparameters:", best_params)


Best Hyperparameters: {'model__normalize': False, 'model__l1_ratio': 0.1, 'model__fit_intercept': True, 'model__alpha': 1.0}


In [57]:
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score

# Calculate regression metrics
mse = mean_squared_error(y_test, y_pred)
mae = mean_absolute_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

print("Mean Squared Error (MSE):", mse)
print("Mean Absolute Error (MAE):", mae)
print("R-squared (R2) Score:", r2)


Mean Squared Error (MSE): 66.8379408140123
Mean Absolute Error (MAE): 6.531756925658417
R-squared (R2) Score: 0.5366361084787465


In [58]:
pipeline = Pipeline([
    ('imputer', SimpleImputer(strategy='mean')),
    ('model', ElasticNet())
])

# Fit the pipeline
pipeline.fit(x_train, y_train)

# Score the model
score = pipeline.score(x_test, y_test)
y_pred = pipeline.predict(x_test)

In [59]:
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score

# Calculate regression metrics
mse = mean_squared_error(y_test, y_pred)
mae = mean_absolute_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

print("Mean Squared Error (MSE):", mse)
print("Mean Absolute Error (MAE):", mae)
print("R-squared (R2) Score:", r2)


Mean Squared Error (MSE): 66.83414936511024
Mean Absolute Error (MAE): 6.520221809651143
R-squared (R2) Score: 0.5366623932579658


In [68]:
from sklearn.model_selection import RandomizedSearchCV

# Define the parameter grid
param_grid = {
  'imputer__strategy': ['mean', 'median', 'most_frequent'],
  'model__n_estimators': [100, 200, 500],
  'model__max_depth': [3, 5, 7],
}

# Create a RandomizedSearchCV object
random_search = RandomizedSearchCV(pipeline, param_grid, cv=5, n_iter=10)

# Fit the RandomizedSearchCV object
random_search.fit(x_train, y_train)

# Get the best model
best_model = random_search.best_estimator_

# Score the best model
score = best_model.score(x_test, y_test)
y_pred = best_model.predict(x_test)


In [71]:
best_model

Pipeline(steps=[('imputer', SimpleImputer()),
                ('model',
                 XGBRegressor(base_score=0.5, booster='gbtree',
                              colsample_bylevel=1, colsample_bynode=1,
                              colsample_bytree=1, enable_categorical=False,
                              gamma=0, gpu_id=-1, importance_type=None,
                              interaction_constraints='',
                              learning_rate=0.300000012, max_delta_step=0,
                              max_depth=7, min_child_weight=1, missing=nan,
                              monotone_constraints='()', n_estimators=500,
                              n_jobs=88, num_parallel_tree=1, predictor='auto',
                              random_state=0, reg_alpha=0, reg_lambda=1,
                              scale_pos_weight=1, subsample=1,
                              tree_method='exact', validate_parameters=1,
                              verbosity=None))])

In [70]:
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score

# Calculate regression metrics
mse = mean_squared_error(y_test, y_pred)
mae = mean_absolute_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

print("Mean Squared Error (MSE):", mse)
print("Mean Absolute Error (MAE):", mae)
print("R-squared (R2) Score:", r2)


Mean Squared Error (MSE): 40.72679959708088
Mean Absolute Error (MAE): 4.777519607916407
R-squared (R2) Score: 0.7109525416978151


In [74]:
pipeline = Pipeline([
    ('imputer', SimpleImputer(strategy='mean')),
    ('model', XGBRegressor(base_score=0.5, booster='gbtree',
                              colsample_bylevel=1, colsample_bynode=1,
                              colsample_bytree=1, enable_categorical=False,
                              gamma=0, gpu_id=-1, importance_type=None,
                              interaction_constraints='',
                              learning_rate=0.300000012, max_delta_step=0,
                              max_depth=7, min_child_weight=1,
                              monotone_constraints='()', n_estimators=500,
                              n_jobs=88, num_parallel_tree=1, predictor='auto',
                              random_state=0, reg_alpha=0, reg_lambda=1,
                              scale_pos_weight=1, subsample=1,
                              tree_method='exact', validate_parameters=1,
                              verbosity=None))])


# Fit the pipeline
pipeline.fit(x_train, y_train)

# Score the model
score = pipeline.score(x_test, y_test)
y_pred = pipeline.predict(x_test)

In [75]:
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score

# Calculate regression metrics
mse = mean_squared_error(y_test, y_pred)
mae = mean_absolute_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

print("Mean Squared Error (MSE):", mse)
print("Mean Absolute Error (MAE):", mae)
print("R-squared (R2) Score:", r2)


Mean Squared Error (MSE): 40.72679959708088
Mean Absolute Error (MAE): 4.777519607916407
R-squared (R2) Score: 0.7109525416978151


In [25]:
from sklearn.ensemble import StackingRegressor
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor, AdaBoostRegressor
from sklearn.linear_model import LinearRegression, Ridge, Lasso
from xgboost import XGBRegressor
from lightgbm import LGBMRegressor
from sklearn.svm import SVR
from sklearn.metrics import mean_squared_error

# Define the base regression models
base_models = [
    ('lr', LinearRegression(n_jobs=-1)),  # Specify n_jobs here
    ('ridge', Ridge(alpha=1.0)),
    ('lasso', Lasso(alpha=1.0)),
    ('rf', RandomForestRegressor(n_estimators=100, random_state=42, n_jobs=-1)),  # Specify n_jobs here
    ('gb', GradientBoostingRegressor(n_estimators=100, random_state=42)),
    ('ada', AdaBoostRegressor(n_estimators=100, random_state=42)),
    ('xgb', XGBRegressor(n_estimators=100, random_state=42, n_jobs=-1)),  # Specify n_jobs here
    ('lgbm', LGBMRegressor(n_estimators=100, random_state=42, n_jobs=-1)),  # Specify n_jobs here
]

# Define the meta-regressor (the model that combines base models)
meta_regressor = SVR()

# Create the stacking ensemble
stacking_regressor = StackingRegressor(estimators=base_models, final_estimator=meta_regressor, n_jobs=-1)  # Specify n_jobs here

# Train the stacking regressor
stacking_regressor.fit(x_train, y_train)

# Make predictions
y_pred = stacking_regressor.predict(x_test)

# Evaluate the performance
mse = mean_squared_error(y_test, y_pred)
print(f"Mean Squared Error: {mse}")


Mean Squared Error: 43.102674324124436


In [26]:
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score

# Calculate regression metrics
mse = mean_squared_error(y_test, y_pred)
mae = mean_absolute_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

print("Mean Squared Error (MSE):", mse)
print("Mean Absolute Error (MAE):", mae)
print("R-squared (R2) Score:", r2)


Mean Squared Error (MSE): 43.102674324124436
Mean Absolute Error (MAE): 4.907699553273428
R-squared (R2) Score: 0.6965601302544786


In [ ]:
stacking_regressor.score(x_train, y_train)

NameError: name 'pipeline' is not defined